# Step Forward Cross Validation for Bioactivity Prediction

## Predicting LogP, LogD and Computing MCE-18 as sorting variables

Here, we predict and add columns for CrippenLogP ([rdkit](https://www.rdkit.org/docs/GettingStartedInPython.html#descriptor-calculation)), LogD ([Code from [Pat Walters](https://github.com/PatWalters)](https://gist.github.com/PatWalters/7aebcd5b87ceb466db91b11e07ce3d21)) and compute [MCE-18](https://pubs.acs.org/doi/abs/10.1021/acs.jmedchem.9b00004) for valid SMILES in
`../benchmark/data/standardized/` and save the results to `../benchmark/data/processed`

In [1]:
import os

import lightgbm as lgbm
import pandas as pd
from rdkit import Chem, RDLogger
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem import rdchem, Descriptors
from tqdm import tqdm

In [2]:
RDLogger.DisableLog("rdApp.*")

In [3]:
os.makedirs('../benchmark/data/processed', exist_ok=True)

### LogD Prediction
**"A LogD model based on 1.8M datapoints from ChEMBL"** -- based on Dr. Patrick Walters's [Gist](https://gist.github.com/PatWalters/7aebcd5b87ceb466db91b11e07ce3d21) ref.
https://practicalcheminformatics.blogspot.com/2022/01/the-solubility-forecast-index.html

In [4]:
class LogDPredictor:
    def __init__(self, model_file_name="./logdmodel.txt"):
        if not os.path.exists(model_file_name):
            raise FileNotFoundError(f"model file not found in {model_file_name}")
        self.mdl = lgbm.Booster(model_file=model_file_name)
        self.descList = self.mdl.feature_name()
        self.fns = [(x, y) for x, y in Descriptors.descList if x in self.descList]

    def predict_smiles(self, smi):
        mol = Chem.MolFromSmiles(smi)
        if mol:
            return self.predict(mol)
        return None

    def predict(self, mol):
        res = []
        for _, y in self.fns:
            res.append(y(mol))
        return self.mdl.predict([res])[0]

In [5]:
def compute_mce18(smiles: str) -> float | None:
    # copypasta from https://github.com/SkylerKramer/MedChem/blob/master/mce18.txt
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    ar = rdmd.CalcNumAromaticRings(mol)
    nar = rdmd.CalcNumAliphaticRings(mol)
    spiro = rdmd.CalcNumSpiroAtoms(mol)
    sp3 = rdmd.CalcFractionCSP3(mol)
    chiral = int(bool(Chem.FindMolChiralCenters(mol, includeUnassigned=True)))

    zagreb = 0
    index = 0
    cyc = 0
    for atom in mol.GetAtoms():
        zagreb = zagreb + rdchem.Atom.GetDegree(atom) ** 2
        if (
                atom.GetAtomicNum() == 6
                and mol.GetAtomWithIdx(index).IsInRing() == True
                and rdchem.Atom.GetHybridization(atom) == 4
        ):
            cyc += 1
        index += 1

    cyc = cyc / mol.GetNumAtoms()
    acyc = sp3 - cyc
    q = 3 - 2 * mol.GetNumAtoms() + zagreb / 2
    return q * (ar + nar + spiro + chiral + (sp3 + cyc - acyc) / (1 + sp3))

In [6]:
pred = LogDPredictor()


def predict_logd(smiles: str) -> float | None:
    return pred.predict_smiles(smiles)

In [7]:
def predict_logp(smiles: str) -> float | None:
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return Descriptors.MolLogP(mol)

In [9]:
for fname in tqdm(os.listdir('../benchmark/data/standardized/'), desc="Compute Sorting Parameter"):
    if fname.endswith('.csv'):
        df = pd.read_csv(f'../benchmark/data/standardized/{fname}')
        df["LogD"] = df["canonical_smiles"].apply(predict_logd)
        df["LogP"] = df["canonical_smiles"].apply(predict_logp)
        df["MCE18"] = df["canonical_smiles"].apply(compute_mce18)
        # Filter rows with NaN and print if any exist
        missing_values_df = df[df[['LogD', 'LogP', 'MCE18']].isna().any(axis=1)]
        if not missing_values_df.empty:
            print(fname, df[df[['LogD', 'LogP', 'MCE18']].isna().any(axis=1)])
            raise Exception("Missing Values in LogD, LogP or MCE18")
        df.to_csv(f'../benchmark/data/processed/{fname}', index=False)

Compute Sorting Parameter: 100%|██████████| 120/120 [07:46<00:00,  3.88s/it]
